# Blend Mask (Wild Dash)

**(Esegui tutto)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyyaml==5.1

# Cerchiamo la versione che porti il match perfetto tra le librerie
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

from IPython.display import Image, clear_output

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

clear_output()

In [ ]:
# Cloning AdelaiDet
%cd /content
!git clone https://github.com/aim-uofa/AdelaiDet.git
%cd AdelaiDet/
!python setup.py build develop

clear_output()

In [ ]:
# Some basic setup:
import adet
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from adet.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
!wget -O blendmask_r101_3x.pth https://cloudstor.aarnet.edu.au/plus/s/e4fXrliAcMtyEBy/download

clear_output()

**Scelta del modello**

In [ ]:
model_BM_ResNet_101 = "configs/BlendMask/R_101_3x.yaml"
weight_BM_ResNet_101 = "blendmask_r101_3x.pth"

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("dash_train", {}, "/content/drive/MyDrive/datasets/wd_public_02/wilddash_train.json", "/content/drive/MyDrive/datasets/wd_public_02/images_train")


from detectron2.data.datasets import register_coco_instances
register_coco_instances("dash_test", {}, "/content/drive/MyDrive/datasets/wd_public_02/wilddash_test.json", "/content/drive/MyDrive/datasets/wd_public_02/images_test")

In [ ]:
from detectron2.engine import DefaultTrainer


cfg = get_cfg()
cfg.merge_from_file(model_BM_ResNet_101)
cfg.DATASETS.TRAIN = ("dash_train",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = weight_BM_ResNet_101
cfg.INPUT.MASK_FORMAT = 'bitmask'


cfg.MODEL.BASIS_MODULE.NORM = "BN" # Not SyncBN because we have only 1 GPU and 1 machine, otherwise there is a problem with init groups of process
cfg.MODEL.BASIS_MODULE.LOSS_ON = False


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.SOLVER.IMS_PER_BATCH = 4 #su mask usiamo 1... 
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1500,)

# Classi di Wild Dash
#cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13
cfg.MODEL.BASIS_MODULE.NUM_CLASSES = 13 # of BlendMask


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("dash_test", cfg, False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "dash_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

**Stampa immagini**

In [ ]:
def print_inference(img_path, predictor):
  img = cv2.imread(img_path)
  outputs = predictor(img)
  v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=0.5)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


# Esempio di utilizzo
print_inference("/content/drive/MyDrive/datasets/wd_public_02/images_test/tk0000_100000.jpg", predictor)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output